# Laboratorio 4 | Machine Learning
* Marco Ferraro | B82957
* Roy Padilla | B85854

In [ ]:
import pandas as pd # procesamiento de dataset
import math # uso de función exp
import numpy as np # uso de funciones y generación de arreglos
import math # operación de raíz cuadrada

LIMIT = 1e-5

In [ ]:
target_value_titanic = ['Survived_0', 'Survived_1']
df_titanic = pd.read_csv('titanic.csv')
df_titanic = df_titanic.drop(columns=['PassengerId', 'Name', 'Ticket', 'Cabin']).dropna()

# Se pasan a one-hot encoding las columnas 'Pclass','Sex' y 'Survived'
df_titanic = pd.get_dummies(df_titanic,columns=['Embarked', 'Pclass','Sex', 'Survived'])
y = df_titanic.loc[:, target_value_titanic]


df_titanic = df_titanic.drop(columns = target_value_titanic)

df_titanic.shape

1. Función `sigmoid(x)` que recibe un número `x` y retorna el resultado de la función sigmoide para dicho `x:   1/(1 + e^(-x))`


In [ ]:
def sigmoid(x):
  try:
    return 1 / (1 + np.exp(-x))
  except OverflowError:
    return LIMIT

2. Función d_sigmoid(y) que recibe un número y (tal que y = sigmoid(x)) y retorna el resultado de la derivada de la función sigmoide para el y dado: y*(1-y)


In [ ]:
def d_sigmoid(y):
  try:
    return y * (1 - y)
  except OverflowError:
    return LIMIT

3. Función `tanh(x)` que recibe un número x y retorna el resultado de la función tangente hiperbólico para dicho `x: (e^x - e^(-x))/(e^x + e^(-x))`

In [ ]:
def tanh(x):
  try:
    return (np.exp(x) - np.exp(-x))/(np.exp(x) + np.exp(-x))
  except OverflowError:
    return LIMIT

4. Función `d_tanh(y)` que recibe un número y (tal que `y = tanh(x)`) y retorna el resultado de la derivada de la función tangente hiperbólico para el `y` dado: `1 - y^2`

In [ ]:
def d_tanh(y):
  try:
    return 1-y**2
  except OverflowError:
    return LIMIT

5. Función `relu(x)` que recibe un número x y retorna el resultado de la función lineal rectificada para dicho `x: (x if x > 0 else 0)`

In [ ]:
def relu(x):
  try:
    return x if x > 0 else 0
  except OverflowError:
    return LIMIT

6. Función `d_relu(y)` que recibe un número `y` (tal que `y = relu(x)`) y retorna el resultado de la derivada de la función lineal rectificada para el y dado: `1 if y >0 else 0`

In [ ]:
def d_relu(y):
  try:
    return 1 if y > 0 else 0
  except OverflowError:
    return LIMIT

7. `Función lrelu(x)` que recibe un número x y retorna el resultado de la función lineal rectificada con fuga para dicho x: (x if x>0 else 0.01x)

In [ ]:
def lrelu(x):
  try:
    return x if x>0 else 0.01*x
  except OverflowError:
    return LIMIT

8. Función `d_lrelu(y)` que recibe un número y (tal que y = lrelu(x)) y retorna el resultado de la derivada de la función lineal rectificada con fuga para el y dado: 1 if y>0 else 0.01

In [ ]:
def d_lrelu(y):
  try:
    return 1 if y > 0 else 0.01
  except OverflowError:
    return LIMIT

`Clase DenseNN`


10. `Método predict(self, x)` que recibe una matriz de datos x de tamaño nxm, donde m coincide con el primer valor de layers al construir la red neuronal. Este método ejecuta el forward propagation aplicando la multiplicación con la matrices de pesos y las funciones de activación de cada capa. Finalmente, retorna una matriz de datos nxp, donde p coincide con el último de layers y corresponde a los valores predichos por la red neuronal para los casos x.

In [ ]:
class DenseNN:
    def __init__(self, layers, activation, seed = 0):
      np.random.seed(seed)

      self.x_count = layers[0]
      self.y_count = layers[-1]
      self.activation = activation
      self.epoch = 0
      self.momentum = 0
      self.lr = 0
      self.decay = 0

      self.weight_matrixs = []
      self.delta_weight_matrix = []
      self.prev_delta_weight_matrix = []
      
      self.net_values = []
      self.activation_values = []
      self.layers_errors = []

      self.activation_values_reduced = []
      self.layers_errors_reduced = []

      self.error_history = []

      # inicialización de la matriz de pesos y bias
      for index in range(len(layers[1:])):
        fan_in = layers[index] # capa anterior (cantidad de columnas)
        fan_out = 0 if len(layers[1:-1]) < index + 1 else layers[index+2] # capa siguiente
        self.weight_matrixs.append(np.random.normal(loc = 0, scale=2.0/math.sqrt(fan_in + fan_out), size = (fan_in + 1, layers[index + 1])))

    def predict(self, x = pd.DataFrame):
      x = x.assign(bias = np.ones(x.shape[0])) # inserción del bias
      predictions = np.array([])
      for index, row in x.iterrows():
        # llamada al forward propagation para la fila
        predictions = np.append(predictions, self._fowardPropagation(row))
      return predictions 

    def train(self, lr=0.05, momentum=0, decay=0):
      self.epoch = 0
      self.error_history = []
      # REVISAR
      self.lr = lr
      self.momentum = momentum
      self.decay = decay
      self.delta_weight_matrix = []
      for weight_matrix in self.weight_matrixs:
        self.delta_weight_matrix.append(np.copy(weight_matrix))
        self.prev_delta_weight_matrix.append(np.copy(weight_matrix))



    def _fowardPropagation(self, row):
      # limpieza de los valores netos, activación y error
      # self.activation_values = []
      self.net_values = []
      
      # inicialización de los valores para la primer capa oculta
      net_k = np.matmul(np.transpose(row.to_numpy()), self.weight_matrixs[0])
      o_k = self._getActivationValue(0, net_k)
      
      self.net_values.append(net_k)
      
      # recorrido desde la segunda capa oculta hasta la salida
      for weight_index in range(len(self.weight_matrixs[1:])):
        # aplicación de la multiplicación: x*w
        net_k = np.matmul(np.transpose(np.append(o_k, 1)), self.weight_matrixs[weight_index + 1])
        # Aplicación de la función de activación en cada uno de los resultados
        o_k = self._getActivationValue(weight_index + 1, net_k)
        self.net_values.append(net_k)

      # Retorna el valor final de la predicción de la tupla
      return o_k

    def backPropagation(self, x, y):
      x = x.assign(bias = np.ones(x.shape[0]))

      self.layers_errors = [np.array([])] * len(self.weight_matrixs)
      self.activation_values = [np.array([])] * len(self.weight_matrixs)

      self.layers_errors_reduced = [np.array([])] * len(self.weight_matrixs)
      self.activation_values_reduced = [np.array([])] * len(self.weight_matrixs)

      row_counter = 1

      for weight_matrix in self.delta_weight_matrix:
        self.prev_delta_weight_matrix.append(np.copy(weight_matrix))

      for index, row in x.iterrows():
        y_predict = self._fowardPropagation(row)
        y_true = y.loc[index]
        dj = []
        for layer_index in reversed(range(len(self.weight_matrixs))):
          if layer_index == len(self.weight_matrixs) - 1: # caso para capa de salida
            dj = 2 * (y_predict - y_true) * self._getDerivativeActivationValues(layer_index, self.net_values[layer_index])
            oj = self._getActivationValue(layer_index, self.net_values[layer_index])

          elif layer_index == 0: # caso para capa de entrada
            dj = (self.weight_matrixs[layer_index + 1] @ dj) * np.append(self._getDerivativeActivationValues(layer_index, self.net_values[layer_index]), 1)
            dj = np.delete(dj, -1) # eliminación del bias
            oj = self.net_values[layer_index]

          else: # caso para capas ocultas
            dj = (self.weight_matrixs[layer_index + 1] @ dj) * np.append(self._getDerivativeActivationValues(layer_index, self.net_values[layer_index]), 1)
            dj = np.delete(dj, -1) # eliminación del bias
            oj = self._getActivationValue(layer_index, self.net_values[layer_index])
          # Generación de matriz de errores
          self.layers_errors[layer_index] = np.append(self.layers_errors[layer_index], np.array(dj))
          self.layers_errors[layer_index] = np.reshape(self.layers_errors[layer_index], (row_counter,dj.shape[0]))
          # Generación de matriz de oj (valores de activación)
          self.activation_values[layer_index] = np.append(self.activation_values[layer_index], np.array(oj))
          self.activation_values[layer_index] = np.reshape(self.activation_values[layer_index], (row_counter,oj.shape[0]))
        # Incremento del row index para el reshape
        row_counter += 1
      

      # Reducir dimensionalidad
      for layer_index in range(len(self.weight_matrixs)):
        for col in range(len(self.layers_errors[layer_index][0])):
          error_mean = np.mean(self.layers_errors[layer_index][:, col])
          activaction_mean = np.mean(self.activation_values[layer_index][:, col])

          self.layers_errors_reduced[layer_index] = np.append(self.layers_errors_reduced[layer_index], error_mean)
          self.activation_values_reduced[layer_index] = np.append(self.activation_values_reduced[layer_index], activaction_mean)

      # apply values to delta w
      for layer_index in reversed(range(len(self.weight_matrixs))):
          if layer_index != 0:
            activation_vec = np.array([self.activation_values_reduced[layer_index - 1]])
            errors_vec = np.array([self.layers_errors_reduced[layer_index]])
            result = activation_vec.T @ errors_vec

            for i in range(len(result)):
              for j in range(len(result[0])):
                self.weight_matrixs[layer_index][i][j] = result[i][j]

      self.error_history.append(self.layers_errors_reduced)

    def step(self):
      for layer_index in range(len(self.weight_matrixs)):
        self.weight_matrixs[layer_index] = self.weight_matrixs[layer_index] - (self.lr *  (self.delta_weight_matrix[layer_index] + self.momentum * self.prev_delta_weight_matrix[layer_index]))

      self.lr = self.lr / (1 + self.decay)
      self.epoch += 1

    def _getActivationValue(self, layer_number, values):
      return_values = []
      for value in np.nditer(values):
        return_values.append(self._applyActivation(layer_number, value))
      return np.array(return_values)

    def _getDerivativeActivationValues(self, layer_number, values):
      return_values = []
      for value in np.nditer(values):
        return_values.append(self._applyActivationDerivative(layer_number, value))
      return np.array(return_values)

    def _applyActivation(self, layer_number, value):
      if self.activation[layer_number] == 's': # aplicar función sigmoide
        return sigmoid(value)
      elif self.activation[layer_number] == 't': # aplicar función tangente hiperbólico
        return tanh(value)
      elif self.activation[layer_number] == 'r': # aplicar función relu
        return relu(value)
      elif self.activation[layer_number] == 'l': # aplicar función leaky relu
        return lrelu(value)
      return 0

    def _applyActivationDerivative(self, layer_number, value):
      if self.activation[layer_number] == 's': # aplicar función sigmoide
        return d_sigmoid(value)
      elif self.activation[layer_number] == 't': # aplicar función tangente hiperbólico
        return d_tanh(value)
      elif self.activation[layer_number] == 'r': # aplicar función relu
        return d_relu(value)
      elif self.activation[layer_number] == 'l': # aplicar función leaky relu
        return d_lrelu(value)
      return 0


In [ ]:
neural_net = DenseNN([12, 3, 2],['s','s'], 0)
neural_net.train()

for i in range(5):
  neural_net.backPropagation(df_titanic, y)
  neural_net.step()

neural_net.error_history

In [ ]:
neural_net = DenseNN([12, 3, 2],['r','r'], 0)
neural_net.train()

for i in range(5):
  neural_net.backPropagation(df_titanic, y)
  neural_net.step()

neural_net.error_history

In [ ]:
neural_net = DenseNN([12, 5, 6, 2],['s', 's','r'], 0)
neural_net.train()

for i in range(5):
  neural_net.backPropagation(df_titanic, y)
  neural_net.step()

neural_net.error_history

¿Se va reduciendo el error? Intente diferentes combinaciones de parámetros, arquitecturas y funciones de activación para lograr que su algoritmo converja. 

¿Cuál combinación le produjo el mejor resultado?

> Como se puede ver en los resultado, los valores de los errores de las neuronas van covergiendo pero para constantes muy altas. Inclusive, para las funciones. Esto se debe a la implementación de limites que se está manejando dentro de las funciones matemáticas. Consideramos que, entre varias configuraciones, la que maneja mejor consistencia es la úlitma que maneja `DenseNN([12, 5, 6, 2],['s', 's','r'], 0)`

¿Qué aprendió de estos experimentos? (Se vale no haber aprendido nada, venimos a
experimentar)

> Dentro de estos experimentos se tuvo un entendimiento más amplio sobre el uso del gradiente del descenso. Además, se aprendió de como se debe mantener un buen registro y cálculo del error para el cálculo de backpropagation.

